In [1]:
import os
import tempfile
import requests
import rasterio
from rasterio.errors import RasterioError
import geopandas as gpd
from shapely.geometry import box, mapping
from shapely import wkt
from pyproj import Transformer
import numpy as np
import logging
import pandas as pd

In [2]:
# load local functions
import sys
sys.path.append('/mnt/c/Users/jmaynard/Documents/GitHub/cu-nrcs/scritps')
import gaez_config
from gaez_config import hz_names
import GAEZ_SSURGO_data
import GAEZ_US_phase_calc
import GAEZ_SQI_functions
import GAEZ_crop_req
import GAEZ_soil_data_processing

In [3]:
# Define the AOI as a WKT polygon in EPSG:4326
aoi_wkt = 'POLYGON((-101.7703 41.1811, -101.7703 41.3042, -101.4972 41.3042, -101.4972 41.1811, -101.7703 41.1811))'
aoi_geom = wkt.loads(aoi_wkt)

# Create a GeoDataFrame with the AOI (EPSG:4326)
aoi = gpd.GeoDataFrame({'geometry': [aoi_geom]}, crs="EPSG:4326")

# For WCS, we need the AOI in the native CRS (EPSG:5070 for gssurgo); the function will do that.
raster_obj = GAEZ_SSURGO_data.mukey_wcs(aoi, db='gssurgo', res=30, quiet=False)

# Read the first band from the raster (assuming mukey values are stored in band 1)
data = raster_obj.read(1)

# If there's a nodata value defined, filter it out
if raster_obj.nodata is not None:
    unique_mukey = np.unique(data[data != raster_obj.nodata])
else:
    unique_mukey = np.unique(data)
mukey_list = unique_mukey.astype(int)
print(mukey_list)

Constructed WCS Request URL:
http://casoilresource.lawr.ucdavis.edu/cgi-bin/mapserv?map=/soilmap2/website/wcs/mukey.map&SERVICE=WCS&VERSION=2.0.1&REQUEST=GetCoverage&COVERAGEID=gSSURGO&FORMAT=image/tiff&GEOTIFF:COMPRESSION=DEFLATE&SUBSETTINGCRS=EPSG:5070&FORMAT=GEOTIFF_FLOAT&SUBSET=x(-479970.4134164839,-456483.3531603016)&SUBSET=y(2032497.2306275005,2047599.635876622)&RESOLUTION=x(30)&RESOLUTION=y(30)
[1698868 1698869 1698871 1698872 1698873 1698874 1698875 1698879 1698880
 1698881 1698882 1698885 1698886 1698887 1698888 1698890 1698891 1698892
 1698893 1698894 1698897 1698899 1698900 1698901 1698903 1698904 1698906
 1698907 1698908 1698909 1698910 1698912 1698915 1698916 1698917 1698919
 1698920 1698925 1698926 1698927 1698928 1698929 2738640 2738668 2738708
 2738727 2837772 2837877 2837878]


In [4]:
ssurgo_df = GAEZ_SSURGO_data.ssurgo_gaez_data(mukey_list)

print(ssurgo_df)
print(ssurgo_df.dtypes)

       mukey     cokey     chkey hzname  hzdept_r  hzdepb_r  sand  silt  clay  \
0    1698868  25328238  75613234      A         0        43  64.6  26.9   8.5   
1    1698868  25328238  75613232    ACk        43        81  69.4  24.1   6.5   
2    1698868  25328238  75613233    2Cg        81       152  92.5   6.5   1.0   
3    1698868  25328239  75613237      A         0         8  79.2  15.8   5.0   
4    1698868  25328239  75613235      C         8        20  95.5   1.5   3.0   
..       ...       ...       ...    ...       ...       ...   ...   ...   ...   
400  2837878  25328475  75614099      C        36       203  96.0   2.0   2.0   
401  2837878  25328476  75614100      A         0        15  80.0  12.0   8.0   
402  2837878  25328476  75614101     AC        15        43  79.0  13.0   8.0   
403  2837878  25328476  75614102     C1        43        69  79.0  13.0   8.0   
404  2837878  25328476  75614103     C2        69       203  79.0  13.0   8.0   

      pi  ...    soc       

In [5]:
ssurgo_phase_df = GAEZ_US_phase_calc.classify_gaez_v4_phases(ssurgo_df)
# ssurgo_phase_df = ssurgo_phase_df.applymap(lambda x: str(x) if isinstance(x, list) else x)
print(ssurgo_phase_df)

       mukey     cokey     chkey hzname  hzdept_r  hzdepb_r  sand  silt  clay  \
0    1698868  25328238  75613234      A         0        43  64.6  26.9   8.5   
1    1698868  25328238  75613232    ACk        43        81  69.4  24.1   6.5   
2    1698868  25328238  75613233    2Cg        81       152  92.5   6.5   1.0   
3    1698868  25328239  75613237      A         0         8  79.2  15.8   5.0   
4    1698868  25328239  75613235      C         8        20  95.5   1.5   3.0   
..       ...       ...       ...    ...       ...       ...   ...   ...   ...   
400  2837878  25328475  75614099      C        36       203  96.0   2.0   2.0   
401  2837878  25328476  75614100      A         0        15  80.0  12.0   8.0   
402  2837878  25328476  75614101     AC        15        43  79.0  13.0   8.0   
403  2837878  25328476  75614102     C1        43        69  79.0  13.0   8.0   
404  2837878  25328476  75614103     C2        69       203  79.0  13.0   8.0   

      pi  ...        db  dr

In [18]:
map_data = ssurgo_phase_df[ssurgo_phase_df['cokey'] == '25328476']
# map_data = ssurgo_phase_df.iloc[[237]]
print(map_data)
print(map_data['drainagecl'])

       mukey     cokey     chkey hzname  hzdept_r  hzdepb_r  sand  silt  clay  \
401  2837878  25328476  75614100      A         0        15  80.0  12.0   8.0   
402  2837878  25328476  75614101     AC        15        43  79.0  13.0   8.0   
403  2837878  25328476  75614102     C1        43        69  79.0  13.0   8.0   
404  2837878  25328476  75614103     C2        69       203  79.0  13.0   8.0   

      pi  ...        db  drain_id  pscl  pscl_id  phase_ids_list  il  swr  \
401  4.0  ...  0.960784       5.0     c        1             [0]   0    0   
402  5.0  ...  0.980392       5.0     c        1             [0]   0    0   
403  7.0  ...  1.039216       5.0     c        1             [0]   0    0   
404  7.0  ...  1.039216       5.0     c        1             [0]   0    0   

     roots  vertic  gelic  
401      0       0      0  
402      0       0      0  
403      0       0      0  
404      0       0      0  

[4 rows x 55 columns]
401    well drained
402    well drained
403  

In [19]:
CROP_ID='4'
depthWt_type = GAEZ_SQI_functions.get_depth_weight_type(CROP_ID)   
print(depthWt_type)

3


In [20]:
map_data
CROP_ID='4' # MAIZE
inputLevel='L'
plot_data=None
site_data=None
lab_data=None

depthWt_type = GAEZ_SQI_functions.get_depth_weight_type(CROP_ID)    
# adjust map_data based on availability of user recorded soil profile data
map_data = GAEZ_soil_data_processing.process_plot_data(plot_data, map_data)
# adjust map_data based on availability of user recorded site data
map_data = GAEZ_soil_data_processing.process_site_data(site_data, map_data)
# adjust map_data based on availability of user recorded soil laboratory data
map_data = GAEZ_soil_data_processing.process_lab_data(lab_data, map_data)
print(map_data)
# ------------------------------------------------------------------------------------
# S0 No constraint (100%)
# S1 Slight constraint (90%)
# S2 Moderate constraint (70%)
# S3 Severe constraint (50%)
# S4 Very severe constraint (30%)
# N  Not suitable (<10%)

# load and query crop requirement tables
crop_reqs = GAEZ_crop_req.getGAEZ_requirements_source(CROP_ID=CROP_ID, inputLevel=inputLevel, source='csv', requirement_type='all')
# Access DataFrames
profile_req = crop_reqs.get("profile")
phase_req = crop_reqs.get("phase")
drainage_req = crop_reqs.get("drainage")
texture_req = crop_reqs.get("texture")
terrain_req = crop_reqs.get("terrain")

# calculate profile depth weights
wts = GAEZ_SQI_functions.calculate_depth_weights(map_data, top_col = hz_names.top_col_name, bottom_col= hz_names.bottom_col_name, depthWt_type=depthWt_type)
print(wts)
# gaez_sqi = GAEZ_SQI_functions.gaez_sqi_ratings(map_data, CROP_ID, inputLevel, depthWt_type=1, plot_data=None, site_data=None, lab_data=None)
# SQI 1
sqi1 = GAEZ_SQI_functions.calculate_SQ1(map_data, profile_req, texture_req, inputLevel, wts)
# SQI 2
sqi2 = GAEZ_SQI_functions.calculate_SQ2(map_data, profile_req, texture_req, inputLevel, wts)
# SQI 3
sqi3 = GAEZ_SQI_functions.calculate_SQ3(map_data, profile_req, texture_req, phase_req, wts)
# SQI 4
sqi4 = GAEZ_SQI_functions.calculate_SQ4(map_data, phase_req, drainage_req)
# SQI 5
sqi5 = GAEZ_SQI_functions.calculate_SQ5(map_data, phase_req, profile_req, wts)
# SQI 6
sqi6 = GAEZ_SQI_functions.calculate_SQ6(map_data, phase_req, profile_req, wts)
# SQI 7
sqi7 = GAEZ_SQI_functions.calculate_SQ7(map_data, phase_req, profile_req, texture_req, wts)

print([sqi1,sqi2,sqi3,sqi4,sqi5,sqi6,sqi7])

       mukey     cokey     chkey hzname  hzdept_r  hzdepb_r  sand  silt  clay  \
401  2837878  25328476  75614100      A         0        15  80.0  12.0   8.0   
402  2837878  25328476  75614101     AC        15        43  79.0  13.0   8.0   
403  2837878  25328476  75614102     C1        43        69  79.0  13.0   8.0   
404  2837878  25328476  75614103     C2        69       203  79.0  13.0   8.0   

      pi  ...        db  drain_id  pscl  pscl_id  phase_ids_list  il  swr  \
401  4.0  ...  0.960784       5.0     c        1             [0]   0    0   
402  5.0  ...  0.980392       5.0     c        1             [0]   0    0   
403  7.0  ...  1.039216       5.0     c        1             [0]   0    0   
404  7.0  ...  1.039216       5.0     c        1             [0]   0    0   

     roots  vertic  gelic  
401      0       0      0  
402      0       0      0  
403      0       0      0  
404      0       0      0  

[4 rows x 55 columns]
Loaded and filtered 'profile' data successful

In [21]:
gaez_sqi = GAEZ_SQI_functions.gaez_sqi_ratings(map_data, CROP_ID, inputLevel, depthWt_type=1, plot_data=None, site_data=None, lab_data=None)
print(gaez_sqi)

Loaded and filtered 'profile' data successfully from CSV.
Loaded and filtered 'texture' data successfully from CSV.
Loaded and filtered 'terrain' data successfully from CSV.
Loaded and filtered 'phase' data successfully from CSV.
Loaded and filtered 'drainage' data successfully from CSV.
       SQ1       SQ2  SQ3    SQ4    SQ5    SQ6  SQ7  SR Input Level     cokey
0  20.0408  94.54641  NaN  100.0  100.0  100.0  NaN NaN           L  25328476
